In [60]:
import pandas as pd
import numpy as np

alpha = 0.01
itr_limit = 2000

dbg = False

In [61]:
"""
z = wx + b
"""
class FullyConnected:
    def __init__(self, out_dim):
        self.out_dim = out_dim
        self.a = None
        self.w = None
        self.b = None
        self.z = None
        self.dw = None
        self.db = None

    def forward(self, a):
        if self.w is None or self.b is None:
            self.w = np.random.random((self.out_dim, a.shape[0]))*0.01
            self.b = np.zeros((self.out_dim, 1))

        self.a = a
        self.z = np.dot(self.w, a) + self.b

        if dbg:
            print("fc_forward: ")
            print(self.z.shape)

        return self.z

    def backward(self, dz):
        m = dz.shape[1]
        self.dw = np.matmul(dz, self.a.T)/m
        self.db = np.sum(dz, axis=1, keepdims=True)/m

        self.w = self.w - alpha*self.dw
        self.b = self.b - alpha*self.db

        da = np.matmul(self.w.T, dz)
        return da


In [62]:
"""
a = ReLU(z)
"""
class ReLU:
    def __init__(self):
        self.out_dim = None
        self.z = None
        self.a = None

    @staticmethod
    def relu(z):
        r = np.maximum(0, z)
        return r

    @staticmethod
    def relu_derivative(z):
        dz = np.array(z, copy=True)
        dz[dz<=0] = 0
        dz[dz>0] = 1
        return dz

    def forward(self, z):
        if self.out_dim is None:
            self.out_dim = z.shape[0]

        self.z = z
        self.a = self.relu(z)

        if dbg:
            print("relu_forward: ")
            print(self.a.shape)

        return self.a

    def backward(self, da):
        dz = np.multiply(da, self.relu_derivative(self.z))
        return dz



In [63]:
def cross_entropy(y_hat, y):
    m = y.shape[1]
    logs = np.multiply(np.log(y_hat),y)
    cost = - np.sum(logs) / m
    return cost

"""
y_hat = e^z/sum(e^x)
"""
class SoftMax:
    def __init__(self):
        self.out_dim = None
        self.z = None
        self.y_hat = None

    def forward(self, z):
        if self.out_dim is None:
            self.out_dim = z.shape[0]

        self.z = z
        self.y_hat = np.exp(z) / np.sum(np.exp(z), axis=0)

        if dbg:
            print("soft_forward: ")
            print(self.y_hat.shape)

        return self.y_hat

    def backward(self, y):
        dz = self.y_hat - y
        return dz

In [64]:
def read_data():
    df = pd.read_csv("Toy Dataset/trainNN.txt", delim_whitespace=True, header=None)
    num_features = df.shape[1] - 1
    df = pd.get_dummies(df, columns=[4], drop_first=False)
    train_dataset = df.to_numpy()
    x_train = train_dataset[:,:num_features]
    y_train = train_dataset[:,num_features:]

    x_train = x_train.T
    y_train = y_train.T

    # print(x_train.shape)
    # print(y_train.shape)
    return x_train, y_train


In [65]:
def run_cnn():
    f1 = open("architecture.txt", "r")
    lines = f1.readlines()
    cnn_layers = list()
    for line in lines:
        words = line.strip().split()
        if words[0].lower() == "fc":
            cnn_layers.append(FullyConnected(int(words[1])))
        elif words[0].lower() == "relu":
            cnn_layers.append(ReLU())
        elif words[0].lower() == "softmax":
            cnn_layers.append(SoftMax())

    f1.close()

    x, y = read_data()

    for itr in range(itr_limit):
        prev_a = x
        for layer in cnn_layers:
            prev_a = layer.forward(prev_a)

        prev_derivative = y
        for i in range(len(cnn_layers)-1,0,-1):
            prev_derivative = cnn_layers[i].backward(prev_derivative)

        if itr % 500 == 0:
            print(cross_entropy(prev_a, y))


    print(prev_a)

In [66]:
"""
run cnn
"""
run_cnn()

1.3879759199347623
1.1636866632742078
1.0622854708411886
0.9857946152906494
[[0.4394747  0.19842716 0.01587462 ... 0.44747162 0.20219324 0.01506944]
 [0.292706   0.28767216 0.11156708 ... 0.29121304 0.28876816 0.10892933]
 [0.17714405 0.29177945 0.32273361 ... 0.17353359 0.28997862 0.32108436]
 [0.09067525 0.22212123 0.54982469 ... 0.08778175 0.21905998 0.55491686]]
